# 🥫 YOLO Can Detection — Lightning Talk Demo

This notebook walks through the full pipeline:

1. **Data Preparation** — explore raw data, split into train/val, visualize annotations
2. **Training** — fine-tune YOLOv8n on our can dataset
3. **Demo Monitor** — live camera inference with Basler GigE camera

---

## 0. Environment Check

In [1]:
import sys, torch, cv2
import ultralytics

print(f"Python:      {sys.version.split()[0]}")
print(f"PyTorch:     {torch.__version__}")
print(f"CUDA:        {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU:         {torch.cuda.get_device_name(0)}")
    print(f"VRAM:        {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
print(f"Ultralytics: {ultralytics.__version__}")
print(f"OpenCV:      {cv2.__version__}")

Python:      3.12.3
PyTorch:     2.6.0+cu124
CUDA:        True
GPU:         NVIDIA GeForce RTX 3060
VRAM:        11.6 GB
Ultralytics: 8.4.16
OpenCV:      4.13.0


### 1.4 Split Dataset (Train / Val)

Split the raw `cans/` data into `dataset/images/{train,val}` and `dataset/labels/{train,val}` with an 80/20 ratio.

This is what `prepare_dataset.py` does — we run it inline here.

In [5]:
import shutil
import random

# Clean old dataset
if Path("dataset").exists():
    shutil.rmtree("dataset")
    print("Removed old dataset/")

# Create directory structure
for split in ["train", "val"]:
    Path(f"dataset/images/{split}").mkdir(parents=True)
    Path(f"dataset/labels/{split}").mkdir(parents=True)

# Shuffle and split
all_images = sorted(Path("cans").glob("*.jpg"))
random.seed(42)  # reproducible split
random.shuffle(all_images)

split_idx = int(0.8 * len(all_images))
splits = {
    "train": all_images[:split_idx],
    "val":   all_images[split_idx:],
}

for split_name, file_list in splits.items():
    for img_file in file_list:
        lbl_file = img_file.with_suffix(".txt")
        shutil.copy2(img_file, f"dataset/images/{split_name}/{img_file.name}")
        if lbl_file.exists():
            shutil.copy2(lbl_file, f"dataset/labels/{split_name}/{lbl_file.name}")

print(f"Train: {len(splits['train'])} images")
print(f"Val:   {len(splits['val'])} images")
print("\nDataset structure:")
for p in sorted(Path("dataset").rglob("*")):
    if p.is_dir():
        count = len(list(p.glob("*")))
        print(f"  {p}/  ({count} files)")

Removed old dataset/
Train: 312 images
Val:   79 images

Dataset structure:
  dataset/images/  (2 files)
  dataset/images/train/  (312 files)
  dataset/images/val/  (79 files)
  dataset/labels/  (2 files)
  dataset/labels/train/  (312 files)
  dataset/labels/val/  (79 files)


### 1.5 Verify `cans.yaml`

YOLO needs a dataset config file pointing to our images.

In [6]:
print(Path("cans.yaml").read_text())

path: ./dataset
train: images/train
val: images/val

nc: 3  # Adjust based on your classes
names: ['0_5L', '0_33L', '0_25L']  # Update with your actual class names



---
## 2. Training

Train **YOLOv8 Nano** from scratch on our can dataset.

| Parameter | Value |
|-----------|-------|
| Base model | `yolov8n.yaml` (random weights, no pretraining) |
| Image size | 640 |
| Epochs | 10 |
| Dataset | `cans.yaml` |

### 2.1 Train from Scratch

Train a YOLOv8 Nano model from random weights (no pretrained COCO weights).

In [1]:
from ultralytics import YOLO

# Build YOLOv8 Nano from scratch (random weights, no pretraining)
model = YOLO("yolov8n.yaml")

# Train on our dataset
results = model.train(data="cans.yaml", epochs=10, imgsz=640)

Ultralytics 8.4.16 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3060, 11909MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=cans.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.yaml, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0

---
## 3. Demo Monitor — Live Camera Inference

The `demo_monitor.py` script runs a live loop:

```
Basler GigE Camera → YOLO Inference (GPU) → OpenCV Display + Side Panel
```

**Controls:**
| Key | Action |
|-----|--------|
| `q` | Quit |
| `s` | Save screenshot |
| `r` | Start/stop recording |
| `c` | Clear FPS stats |

> ⚠️ Requires a connected **Basler GigE camera** and the `pypylon` package.  
> The cell below launches the monitor as a subprocess so the notebook stays responsive.

### 3.1 Install pypylon (camera SDK)

In [4]:
!pip install pypylon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.4 MB/s  0:00:08m0:00:0100:01


### 3.2 Launch Demo Monitor

This opens an OpenCV window with live detections.  
Press **q** in the OpenCV window to stop.

In [10]:
!python demo_monitor.py

Can Detector Monitor
Loading model: best_model/best.pt
Model loaded (GPU)
Camera: acA1300-60gm
Camera ready
Controls: 'q' quit | 's' screenshot | 'r' record | 'c' clear stats
QFontDatabase: Cannot find font directory /home/viktor/lightning_talks_demo/venv/lib/python3.12/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/viktor/lightning_talks_demo/venv/lib/python3.12/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/viktor/lightning_talks_demo/venv/lib/python3.12/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/viktor/lightning_talks_demo/venv/